In [21]:
import numpy as np
import nltk
import pandas as pd
from datasets import load_dataset
import re
import string
from bs4 import BeautifulSoup
import sklearn
#import spacy

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\color\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\color\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\color\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
dataset = load_dataset('artem9k/ai-text-detection-pile')
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'id', 'text'],
        num_rows: 1392522
    })
})

In [23]:
df = pd.DataFrame.from_dict(dataset['train'])
df.head()

,source,id,text
0,human,0,12 Years a Slave: An Analysis of the Film Essa...
1,human,1,20+ Social Media Post Ideas to Radically Simpl...
2,human,2,2022 Russian Invasion of Ukraine in Global Med...
3,human,3,533 U.S. 27 (2001) Kyllo v. United States: The...
4,human,4,A Charles Schwab Corporation Case Essay\n\nCha...


## Preprocessing

In [24]:
# functions for preprocessing
def remove_urls(text):
    return re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", " ", text) # regex taken from https://www.geeksforgeeks.org/python-check-url-string/

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_extra_whitespace(text):
    text = text.strip()
    text = " ".join(text.split())
    return text

def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words("english")
    tokens = [token for token in tokens if token not in stopwords]
    return " ".join(tokens)

def lemmatizer(text):
    tokens = nltk.word_tokenize(text)
    l = nltk.stem.WordNetLemmatizer()
    tokens = [l.lemmatize(token) for token in tokens]
    return " ".join(tokens)

def tokenize_pre_process(text): # for preprocessing using this link: https://spotintelligence.com/2022/12/21/nltk-preprocessing-pipeline/
    # tokenize
    tokens = nltk.word_tokenize(text)

    # remove stop words
    stopwords = nltk.corpus.stopwords.words("english")
    tokens = [token for token in tokens if token not in stopwords]

    # remove top 10% most frequent words 
    fdist = nltk.FreqDist(tokens)
    tokens = [token for token in tokens if fdist[token] < fdist.N() * 0.1]

    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # eliminate punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    return tokens

In [25]:
def preprocess_text(text):
    # encoding to ascii
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # convert text to lower case
    text = text.lower()

    # remove html tags 
    text = remove_html(text)

    # remove urls 
    text = remove_urls(text)

    # remove extra whitespace
    text = remove_extra_whitespace(text)

    # remove stop words
    text = remove_stop_words(text)

    return text

In [26]:
def preprocess_text2(text):
    # encoding to ascii
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # convert text to lower case
    text = text.lower()

    # remove html tags 
    text = remove_html(text)

    # remove urls 
    text = remove_urls(text)

    # remove extra whitespace
    text = remove_extra_whitespace(text)

    # remove stop words
    text = remove_stop_words(text)

    # lemmatize words
    text = lemmatizer(text)
    
    return text

## Feature Engineering

### TFIDF and Count Vectorizer

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
vec = CountVectorizer(max_df=0.9,min_df=0.1)
X = vec.fit_transform(df.text[:1000])

In [29]:
vec.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'about', 'above', 'access',
       'according', 'achieve', 'across', 'act', 'action', 'actions',
       'activities', 'activity', 'addition', 'additional', 'additionally',
       'address', 'affect', 'affected', 'affects', 'after', 'against',
       'age', 'al', 'all', 'allow', 'allowed', 'allows', 'almost',
       'already', 'also', 'although', 'always', 'america', 'american',
       'americans', 'among', 'an', 'analysis', 'another', 'any',
       'approach', 'approaches', 'appropriate', 'are', 'area', 'areas',
       'around', 'article', 'aspect', 'aspects', 'associated', 'at',
       'attention', 'author', 'available', 'avoid', 'back', 'based',
       'basis', 'be', 'became', 'because', 'become', 'becomes', 'been',
       'before', 'behavior', 'being', 'believe', 'benefits', 'best',
       'better', 'between', 'black', 'body', 'both', 'business', 'but',
       'by', 'can', 

In [30]:
vec2 = CountVectorizer(preprocessor=preprocess_text,max_df=0.9,min_df=0.1)
X2 = vec2.fit_transform(df.text[:1000])

In [31]:
vec2.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'access', 'according',
       'achieve', 'across', 'act', 'action', 'actions', 'activities',
       'activity', 'addition', 'additional', 'additionally', 'address',
       'affect', 'affected', 'affects', 'age', 'al', 'allow', 'allowed',
       'allows', 'almost', 'already', 'also', 'although', 'always',
       'america', 'american', 'americans', 'among', 'analysis', 'another',
       'approach', 'approaches', 'appropriate', 'area', 'areas', 'around',
       'article', 'aspect', 'aspects', 'associated', 'attention',
       'author', 'authors', 'available', 'avoid', 'back', 'based',
       'basis', 'became', 'become', 'becomes', 'behavior', 'believe',
       'benefits', 'best', 'better', 'body', 'business', 'care', 'case',
       'cases', 'cause', 'caused', 'causes', 'central', 'century',
       'certain', 'challenges', 'change', 'changes', 'characteristics',
       'children',

In [32]:
vec3 = CountVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1)
X3 = vec3.fit_transform(df.text[:1000])

In [33]:
vec3.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'access', 'according',
       'account', 'achieve', 'across', 'act', 'action', 'activity',
       'addition', 'additional', 'additionally', 'address', 'advantage',
       'affect', 'affected', 'age', 'aim', 'al', 'allow', 'allowed',
       'allows', 'almost', 'already', 'also', 'although', 'always',
       'america', 'american', 'among', 'amount', 'analysis', 'another',
       'application', 'approach', 'appropriate', 'area', 'around',
       'article', 'aspect', 'assessment', 'associated', 'attention',
       'attitude', 'author', 'authority', 'available', 'avoid', 'back',
       'background', 'based', 'basis', 'became', 'become', 'becomes',
       'behavior', 'being', 'belief', 'believe', 'benefit', 'best',
       'better', 'black', 'body', 'book', 'business', 'care', 'case',
       'cause', 'caused', 'center', 'central', 'century', 'certain',
       'challenge', 'chance', '

In [34]:
vec4 = CountVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1, ngram_range=(2,3))
X4 = vec4.fit_transform(df.text[:1000])
vec4.get_feature_names_out()

array(['al 2020', 'content introduction', 'covid 19', 'essay table',
       'essay table content', 'et al', 'et al 2020', 'research paper',
       'table content', 'table content introduction', 'united state',
       'work cited'], dtype=object)

## Doc2Vec

In [35]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [112]:
def doc2vec(data):
    '''
    https://www.geeksforgeeks.org/doc2vec-in-nlp/
    '''
    
    tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()),
                              tags=[str(i)]) for i,doc in enumerate(data)]
    # train the Doc2vec model
    model = Doc2Vec(vector_size=20,
                    min_count=2, epochs=50)
    model.build_vocab(tagged_data)
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
     
    # get the document vectors
    document_vectors = [model.infer_vector(
        word_tokenize(doc.lower())) for doc in data]

    return document_vectors

In [146]:
# Reporting the proportion of samples that are ai generated
print('Percent ai:', round(df[df['source'] == 'ai'].shape[0]/df[df['source'] == 'human'].shape[0]*100, 3))

# Taking a stratified sample of 0.1% of the data
# maintaining same proportions of human and ai samples
data = df.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.001, random_state=0))

Percent ai: 35.44


C:\Users\color\AppData\Local\Temp\ipykernel_29864\875810769.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.001, random_state=0))


In [148]:
preprocess_text2(data.text)

AttributeError: 'Series' object has no attribute 'encode'

## Naive Bayes

In [131]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import scipy.stats as stats

In [135]:
def CI(metric, confidence):
    a,b = stats.t.interval(confidence, 
                         len(metric)-1, 
                         loc=metric.mean(), 
                         scale=metric.std(ddof=1)/np.sqrt(len(metric)))
    return a,b

In [144]:
def evaluation(X_train, X_test, t_train, t_test, model, confidence=0.95, scoring='accuracy'):

    y_train = gnb.predict(X_train)
    y_test = gnb.predict(X_test)
    
    scores = cross_val_score(model,
                             X_train, 
                             t_train, 
                             scoring=scoring, 
                             cv=KFold(10, shuffle=True, random_state=0))
    
    a,b = CI(scores, confidence)
    
    print('===================Naive Bayes Performance=====================')
    print('95% CI = [', a, b, ']')
    print('Train: ', classification_report(t_train, y_train))
    print('Test: ', classification_report(t_test, y_test))

In [142]:
# Vectorize the documents (using non-preprocessed data)
v = doc2vec(data['text'])

# Set up the data and labels
X = np.array(v)
t = data.source
d = {'human' : 0, 'ai' : 1}
t = t.map(d, na_action='ignore')
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=0)

# Naive Bayes Classifier
gnb = GaussianNB()
gnb.fit(X_train, t_train)

GaussianNB()

In [145]:
evaluation(X_train, X_test, t_train, t_test, gnb)

===================Naive Bayes Performance=====================
95% CI = [ 0.7357685953367065 0.7844534123853012 ]
Train:                precision    recall  f1-score   support

           0       0.83      0.86      0.85       827
           1       0.56      0.49      0.52       286

    accuracy                           0.77      1113
   macro avg       0.69      0.68      0.68      1113
weighted avg       0.76      0.77      0.76      1113

Test:                precision    recall  f1-score   support

           0       0.80      0.86      0.83       201
           1       0.56      0.45      0.50        78

    accuracy                           0.75       279
   macro avg       0.68      0.65      0.66       279
weighted avg       0.73      0.75      0.74       279

